In [ ]:
!wget https://storage.googleapis.com/crowdstf-rico-uiuc-4540/rico_dataset_v0.1/unique_uis.tar.gz

--2024-01-17 13:54:31--  https://storage.googleapis.com/crowdstf-rico-uiuc-4540/rico_dataset_v0.1/unique_uis.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.4.207, 172.253.118.207, 74.125.200.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.4.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6471262799 (6.0G) [application/x-tar]
Saving to: ‘unique_uis.tar.gz’

unique_uis.tar.gz   100%[===================>]   6.03G  20.6MB/s    in 5m 6s   

2024-01-17 13:59:37 (20.2 MB/s) - ‘unique_uis.tar.gz’ saved [6471262799/6471262799]



In [ ]:
! apt install -y pigz

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  pigz
0 upgraded, 1 newly installed, 0 to remove and 24 not upgraded.
Need to get 63.6 kB of archives.
After this operation, 162 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 pigz amd64 2.6-1 [63.6 kB]
Fetched 63.6 kB in 1s (88.3 kB/s)
Selecting previously unselected package pigz.
(Reading database ... 121654 files and directories currently installed.)
Preparing to unpack .../archives/pigz_2.6-1_amd64.deb ...
Unpacking pigz (2.6-1) ...
Setting up pigz (2.6-1) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
!pip install -q datasets transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.0 MB/s eta 0:00:00


In [ ]:
from tqdm.autonotebook import tqdm
import os
import pandas as pd
import shutil
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
from huggingface_hub import notebook_login
import torch

<ipython-input-4-39286a02faf7>:1: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
notebook_login()

In [ ]:
%%time
! pigz -dc unique_uis.tar.gz | tar -xf -

CPU times: user 1.21 s, sys: 143 ms, total: 1.35 s
Wall time: 3min 36s


In [ ]:
directory = "./combined"
done_directory = "./annotated"
os.mkdir(done_directory)

In [ ]:
image_filenames = [filename for filename in os.listdir(directory) if "jpg" in filename]

In [ ]:
annotations = []

In [ ]:
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to("cuda")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

In [12]:
for filename in tqdm(image_filenames):
  f = os.path.join(directory, filename)
  raw_image = Image.open(f).convert('RGB')
  text = "a mobile screen showing"
  inputs = processor(raw_image, text, return_tensors="pt").to("cuda", torch.float16)
  out = model.generate(**inputs)
  caption = processor.decode(out[0], skip_special_tokens=True)
  annotations.append([filename,caption])
  shutil.copyfile(f, os.path.join(done_directory, filename))

  0%|          | 0/66261 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [13]:
annotations_df = pd.DataFrame(annotations,columns=["file_name","text"])

In [14]:
annotations_df.shape

(66261, 2)

In [15]:
annotations_df.to_csv(os.path.join(done_directory,"metadata.csv"),index=False)

In [16]:
from datasets import load_dataset

In [17]:
dataset = load_dataset("imagefolder", data_dir=done_directory, split="train")

Resolving data files:   0%|          | 0/66262 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [18]:
dataset.push_to_hub("mowoe/rico-captions-blip-large-conditioned", private=True)

Uploading the dataset shards:   0%|          | 0/15 [00:00<?, ?it/s]

Map:   0%|          | 0/4418 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/45 [00:00<?, ?ba/s]

Map:   0%|          | 0/4418 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/45 [00:00<?, ?ba/s]

...

Creating parquet from Arrow format:   0%|          | 0/45 [00:00<?, ?ba/s]

Map:   0%|          | 0/4417 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/45 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/321 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mowoe/rico-captions-blip-large-conditioned/commit/c46b829fca07f61f3121d75ef35e436a1bbf1e17', commit_message='Upload dataset', commit_description='', oid='c46b829fca07f61f3121d75ef35e436a1bbf1e17', pr_url=None, pr_revision=None, pr_num=None)